## PBL: Huấn luyện Mô hình Nhận diện Hành vi trên Kaggle

### Cell 1: Cài đặt Thư viện & Imports

In [ ]:
!pip install opencv-python-headless

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torchvision.transforms as transforms

import cv2
import os
import numpy as np
import glob
import time
import json
import zipfile
from PIL import Image

from tqdm.notebook import tqdm
from sklearn.metrics import classification_report, accuracy_score

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

### Cell 2: Giải nén Dữ liệu (Chỉ chạy 1 lần)

Tải file `data_clips.zip` lên Kaggle Datasets và kết nối vào Notebook này.

In [ ]:
KAGGLE_INPUT_DIR = "/kaggle/input/pbl-ucf-crime-clips" # <-- SỬA TÊN NÀY
KAGGLE_WORKING_DIR = "/kaggle/working/" # Đây là nơi có thể ghi file

ZIP_PATH = os.path.join(KAGGLE_INPUT_DIR, "data_clips.zip")
EXTRACT_PATH = os.path.join(KAGGLE_WORKING_DIR, "data") # Giải nén vào /kaggle/working/data

# Chỉ giải nén nếu thư mục chưa tồn tại
if not os.path.exists(EXTRACT_PATH):
    print(f"Đang giải nén {ZIP_PATH} vào {EXTRACT_PATH}...")
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_PATH)
    print("Giải nén hoàn tất!")
else:
    print(f"Thư mục {EXTRACT_PATH} đã tồn tại. Bỏ qua giải nén.")

### Cell 3: Định nghĩa Kiến trúc Mô hình (CnnRnn)

In [ ]:
class CnnRnn(nn.Module):
    def __init__(self, num_classes, hidden_size=256, num_layers=2, dropout=0.5):
        super(CnnRnn, self).__init__()

        # --- Phần CNN (ResNet50) ---
        resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        modules = list(resnet.children())[:-1]
        self.resnet = nn.Sequential(*modules)
        for param in self.resnet.parameters():
            param.requires_grad = False

        # --- Phần RNN (LSTM) ---
        self.lstm = nn.LSTM(
            input_size=resnet.fc.in_features, 
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True, 
            dropout=dropout
        )

        # --- Lớp phân loại cuối cùng ---
        self.fc = nn.Linear(hidden_size, num_classes)
        self.dropout_layer = nn.Dropout(dropout)

    def forward(self, x):
        batch_size, seq_length, c, h, w = x.shape
        cnn_in = x.view(batch_size * seq_length, c, h, w)
        
        with torch.no_grad():
            cnn_out = self.resnet(cnn_in)
        
        cnn_out = cnn_out.view(batch_size, seq_length, -1)
        lstm_out, _ = self.lstm(cnn_out)
        last_hidden_state = lstm_out[:, -1, :]
        out = self.dropout_layer(last_hidden_state)
        out = self.fc(out)
        return out

### Cell 4: Định nghĩa `VideoDataset` (An toàn)

In [ ]:
class VideoDataset(Dataset):
    def __init__(self, data_dir, sequence_length=20, transform=None):
        self.data_dir = data_dir
        self.sequence_length = sequence_length
        self.transform = transform
        
        self.classes = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        
        self.video_paths = [] 
        for cls_name in self.classes:
            class_dir = os.path.join(data_dir, cls_name)
            class_idx = self.class_to_idx[cls_name]
            for video_file in os.listdir(class_dir):
                if video_file.endswith(('.mp4', '.avi', '.x264')):
                    self.video_paths.append((os.path.join(class_dir, video_file), class_idx))
                
        print(f"Đã tìm thấy {len(self.video_paths)} video trong '{data_dir}'.")
        print(f"Có {len(self.classes)} lớp: {self.classes}")

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        video_path, label = self.video_paths[idx]
        
        frames = []
        cap = cv2.VideoCapture(video_path)
        
        if not cap.isOpened():
            print(f"Lỗi: Không thể mở video {video_path}. Sẽ trả về tensor đen.")
            cap.release()
            return torch.zeros(self.sequence_length, 3, 224, 224), label

        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        if total_frames <= 0:
            print(f"Cảnh báo: Video {video_path} có 0 frame. Sẽ trả về tensor đen.")
            cap.release()
            return torch.zeros(self.sequence_length, 3, 224, 224), label
        
        frame_indices = np.linspace(0, total_frames - 1, self.sequence_length, dtype=int)
        
        for i in frame_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                pil_img = Image.fromarray(frame)
                if self.transform:
                    frame_tensor = self.transform(pil_img)
                frames.append(frame_tensor)
            else:
                if len(frames) > 0:
                    frames.append(frames[-1]) 
                else:
                    frames.append(torch.zeros(3, 224, 224))

        cap.release()
        video_tensor = torch.stack(frames)
        return video_tensor, label

### Cell 5: Thiết lập Hyperparameters & Đường dẫn Kaggle

In [ ]:
# --- Tham số ---
SEQUENCE_LENGTH = 20  
BATCH_SIZE = 32
NUM_EPOCHS = 25 
LEARNING_RATE = 0.0001

# --- Đường dẫn (Kaggle) ---
TRAIN_DIR = '/kaggle/working/data/train' 
TEST_DIR = '/kaggle/working/data/test'

# Dùng để LƯU KHI ACC CAO NHẤT
BEST_CHECKPOINT_PATH = '/kaggle/working/pbl_best_model.pth' 
# Dùng để TIẾP TỤC NẾU BỊ NGẮT (luôn ghi đè)
LATEST_CHECKPOINT_PATH = '/kaggle/working/pbl_latest_checkpoint.pth' 
# File model cuối cùng (sạch)
MODEL_SAVE_PATH = '/kaggle/working/pbl_final_model.pth' 
HISTORY_SAVE_PATH = '/kaggle/working/training_history.json'
# -----------------------------------------------

# --- Thiết bị ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

### Cell 6: Tạo `Dataset` và `DataLoader`

In [ ]:
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

print("Đang tải Train dataset...")
train_dataset = VideoDataset(
    data_dir=TRAIN_DIR, 
    sequence_length=SEQUENCE_LENGTH, 
    transform=data_transform
)

print("\nĐang tải Test dataset...")
test_dataset = VideoDataset(
    data_dir=TEST_DIR, 
    sequence_length=SEQUENCE_LENGTH, 
    transform=data_transform
)

train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=2
)

test_loader = DataLoader(
    test_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=2
)

NUM_CLASSES = len(train_dataset.classes)
CLASS_NAMES = train_dataset.classes
print(f"\nSố lượng lớp phát hiện: {NUM_CLASSES}")
print(f"Tên các lớp: {CLASS_NAMES}")

### Cell 7: Khởi tạo Mô hình, Loss, Optimizer & Checkpointing

In [ ]:
# Khởi tạo mô hình
model = CnnRnn(num_classes=NUM_CLASSES).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Khởi tạo Mixed Precision (AMP) - Cú pháp đúng
scaler = torch.amp.GradScaler()

#Logic Checkpointing
start_epoch = 0 
best_accuracy = 0.0

# Chúng ta tải từ file LATEST (mới nhất) để tiếp tục
if os.path.exists(LATEST_CHECKPOINT_PATH): 
    print(f"Phát hiện checkpoint MỚI NHẤT! Đang tải từ '{LATEST_CHECKPOINT_PATH}'...")
    try:
        # Tải checkpoint mới nhất
        checkpoint = torch.load(LATEST_CHECKPOINT_PATH) 
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        
        if 'scaler_state_dict' in checkpoint:
            scaler.load_state_dict(checkpoint['scaler_state_dict'])
            
        if 'best_accuracy' in checkpoint:
             # Lấy lại thông tin best_accuracy đã lưu
            best_accuracy = checkpoint['best_accuracy'] 
            
        print(f"Đã tải thành công. Sẽ tiếp tục từ Epoch {start_epoch}. Best Acc đã biết: {best_accuracy:.2f}%")
    except Exception as e:
        print(f"Lỗi khi tải checkpoint: {e}. Bắt đầu huấn luyện lại từ đầu.")
        start_epoch = 0
        best_accuracy = 0.0
else:
    print("Không tìm thấy checkpoint. Bắt đầu huấn luyện từ đầu.")

### Cell 8: Vòng lặp Huấn luyện (Training Loop)

In [ ]:
print("Bắt đầu huấn luyện...")
start_time = time.time()

history = {
    'train_loss': [],
    'test_loss': [],
    'test_acc': []
}

for epoch in range(start_epoch, NUM_EPOCHS):
    epoch_start_time = time.time()
    
    # --- PHA HUẤN LUYỆN ---
    model.train()
    running_loss = 0.0
    
    # Bọc train_loader bằng tqdm
    train_progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]", unit="batch")
    
    for videos, labels in train_progress_bar:
        videos = videos.to(device)
        labels = labels.to(device)
        
        # Cú pháp AMP đúng
        with torch.amp.autocast(device_type='cuda'):
            outputs = model(videos)
            loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        running_loss += loss.item()
        # Cập nhật loss lên thanh progress bar
        train_progress_bar.set_postfix(loss=f"{loss.item():.4f}")

    avg_train_loss = running_loss / len(train_loader)
    history['train_loss'].append(avg_train_loss)
    
    # --- PHA ĐÁNH GIÁ (TESTING) ---
    model.eval()
    test_loss = 0.0
    all_labels = [] # Thu thập nhãn thật
    all_predictions = [] # Thu thập nhãn dự đoán
    
    # Bọc test_loader bằng tqdm
    test_progress_bar = tqdm(test_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Test]", unit="batch")
    
    with torch.no_grad():
        for videos, labels in test_progress_bar:
            videos = videos.to(device)
            labels = labels.to(device)
            
            with torch.amp.autocast(device_type='cuda'):
                outputs = model(videos)
                loss = criterion(outputs, labels) # Tính Test Loss
            
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            
            # Thu thập tất cả dự đoán và nhãn
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            
            test_progress_bar.set_postfix(loss=f"{loss.item():.4f}")

    # --- TÍNH TOÁN CÁC CHỈ SỐ CHI TIẾT ---
    avg_test_loss = test_loss / len(test_loader)
    accuracy = accuracy_score(all_labels, all_predictions) * 100
    
    history['test_loss'].append(avg_test_loss)
    history['test_acc'].append(accuracy)
    
    epoch_time = time.time() - epoch_start_time
    
    # --- IN BÁO CÁO CHI TIẾT ---
    print(f"\n*** === KẾT QUẢ EPOCH [{epoch+1}/{NUM_EPOCHS}] === ***")
    print(f"  Thời gian: {epoch_time:.2f}s")
    print(f"  Average Train Loss: {avg_train_loss:.4f}")
    print(f"  Average Test Loss:  {avg_test_loss:.4f}") 
    print(f"  Test Accuracy:      {accuracy:.2f} %")
    
    # In Báo cáo Phân loại (Precision, Recall, F1-score)
    print("\n--- Báo cáo Chi tiết (Test Set) ---")
    print(classification_report(all_labels, all_predictions, target_names=CLASS_NAMES, digits=3, zero_division=0))
    
    # 1. LƯU CHECKPOINT MỚI NHẤT (GHI ĐÈ)
    # Luôn lưu file này để có thể tiếp tục (resume)
    print(f"  Đang lưu checkpoint MỚI NHẤT (ghi đè) vào '{LATEST_CHECKPOINT_PATH}'...")
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scaler_state_dict': scaler.state_dict(),
        'best_accuracy': best_accuracy, # Lưu lại best_acc hiện tại
    }, LATEST_CHECKPOINT_PATH)

    # 2. LƯU CHECKPOINT TỐT NHẤT (CÓ ĐIỀU KIỆN)
    # Chỉ lưu file này nếu accuracy được cải thiện
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        print(f"  ĐẠT ACCURACY CAO NHẤT MỚI: {accuracy:.2f} %")
        print(f"  Đang lưu checkpoint TỐT NHẤT vào '{BEST_CHECKPOINT_PATH}'...")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'best_accuracy': best_accuracy,
        }, BEST_CHECKPOINT_PATH)
    else:
        print(f"  Accuracy {accuracy:.2f} % (Không cao hơn {best_accuracy:.2f} %). Không ghi đè model TỐT NHẤT.")
    
    print(f'************************************\n')

total_time = time.time() - start_time
print(f"Hoàn thành huấn luyện! Tổng thời gian: {total_time // 60:.0f} phút {total_time % 60:.0f} giây")

### Cell 9: Lưu Model Tốt nhất (Final) & Lịch sử

Sau khi train xong, chúng ta tải checkpoint tốt nhất lên và lưu nó thành file model cuối cùng.

In [ ]:
import json # Thêm import json để cell chạy độc lập

#Tải checkpoint TỐT NHẤT để lưu file cuối
print(f"Đang tải checkpoint TỐT NHẤT từ: {BEST_CHECKPOINT_PATH}")

# Tải lại checkpoint tốt nhất (để đảm bảo)
model = CnnRnn(num_classes=NUM_CLASSES).to(device)
try:
    # Tải checkpoint TỐT NHẤT
    checkpoint = torch.load(BEST_CHECKPOINT_PATH, map_location=device) 
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Tải thành công model tốt nhất (Epoch {checkpoint['epoch']}, Acc: {checkpoint['best_accuracy']:.2f}%)")
    
    # Lưu lại model cuối cùng (chỉ là state_dict)
    torch.save(model.state_dict(), MODEL_SAVE_PATH)
    print(f"Đã lưu model cuối cùng (sạch) vào: {MODEL_SAVE_PATH}")
except Exception as e:
    print(f"Lỗi: Không thể tải/lưu model cuối cùng. File BEST_CHECKPOINT có thể chưa tồn tại. Lỗi: {e}")
# -----------------------------------------------

# Lưu file lịch sử (history) để vẽ biểu đồ
try:
    with open(HISTORY_SAVE_PATH, 'w') as f:
        json.dump(history, f)
    print(f"Đã lưu lịch sử huấn luyện vào: {HISTORY_SAVE_PATH}")
except Exception as e:
    print(f"Lỗi khi lưu file history: {e}")

### Cell 10: Vẽ Biểu đồ

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 6))

# Vẽ Training Loss VÀ Test Loss
plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Training Loss', color='blue')
plt.plot(history['test_loss'], label='Test Loss', color='green')
plt.title('Training & Test Loss qua các Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Vẽ Test Accuracy
plt.subplot(1, 2, 2)
plt.plot(history['test_acc'], label='Test Accuracy', color='orange')
plt.title('Test Accuracy qua các Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig("training_charts.png") # Lưu biểu đồ ra file
plt.show()